In [4]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

In [2]:
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [5]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

In [6]:
tools = [add, multiply, divide]
llm_with_tools = llm.bind_tools(tools)

sys_msg = SystemMessage(
    content="Use either tools to perform arithmetic on a set of inputs or just give me answer with your knowledge")


In [7]:
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

In [8]:
# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")
builder.add_edge("assistant", END)

In [9]:
react_graph = builder.compile()

In [10]:
messages = [HumanMessage(content="Add 3 and 4")]
messages = react_graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4
================================== Ai Message ==================================
Tool Calls:
  add (7cd5082f-2ef7-432f-b6fe-716c4d097f03)
 Call ID: 7cd5082f-2ef7-432f-b6fe-716c4d097f03
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is 7.


In [11]:
messages = [HumanMessage(content="Multiply that by 10")]
messages = react_graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Multiply that by 10
================================== Ai Message ==================================

[{'type': 'text', 'text': 'What number would you like to multiply by 10?', 'extras': {'signature': 'CosIAdHtim/B1WRm9gfrJxEb4s7VH2lBcR/d9vwE0+k3VinPziygrudez6FM3Mmv3B74LnzW3WiLoIEpu/MpUZcqi1xrKs6giftPYpUInbBaRKBeti3+qYlJ1MSVEwgBInSM61J+sm8aNKRit1vEUyljEcPl8cXcV9g+aoDwcf/+aQ/Sd5jf3IrWnS9PRbrXlpfEXrVya59EeNVJuvMG9GTtShfXy/O20pEJziMDlPPmtE2AZLOXoXQjgrYfxVPENb0qX+TENREK3pC1rrtthpKZASxviFGCDVO01xSVPS7cUT6BOIFXRtwTuTUDzbJ9wCHez5wZZ/m8XzENPmAHvehNXdndsLiTcmjt1GUxeyOvtniHbb1FxhMww4qJeaOE6m2ua6r9QiyO/CMBWEEitS+d2GMOAPF/rJ9wvRocUDt3GKNLUJFOhuoFKWlfIOV/x7ltVtDlD9kE73IfCDfH/9ffFbC514zHvOi5882BrwI6XIRJcprYeNB+QrK/WgtJyI5Nu+gUNrcXrW9dKcAPa8FlTigxR2DgPjT71+ePdbj9IDon5nm5jQiPb/PUNHPJW9ZCf72HO2RMUO7U86AVWR1kQOaS9CZdyeoaS70T2+NwvgHTCPz0uhYpiXOZ+0/amk7onOWGZC2u6RjH8Hd5q+iZOheRrSmgi12B8VO+//KRKdiE1r9lNIsxYi4zwFLhUYxGdVup+z3t